<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Разбиение-на-выборки" data-toc-modified-id="Разбиение-на-выборки-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Разбиение на выборки</a></span></li><li><span><a href="#Выбор-наилучшей-модели-и-поиск-гиперпараметров-через-GridSearch" data-toc-modified-id="Выбор-наилучшей-модели-и-поиск-гиперпараметров-через-GridSearch-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Выбор наилучшей модели и поиск гиперпараметров через GridSearch</a></span></li><li><span><a href="#Модель-CatBoost" data-toc-modified-id="Модель-CatBoost-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Модель CatBoost</a></span></li><li><span><a href="#Модель-LightGBM" data-toc-modified-id="Модель-LightGBM-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Модель LightGBM</a></span></li></ul></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [14]:
## импорт библиотек
import numpy as np
import pandas as pd
import torch
import transformers
import tensorflow
import lightgbm as lgb


from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from transformers import BertTokenizer
from pytorch_pretrained_bert import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import make_pipeline, Pipeline
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from wordcloud import WordCloud
from transformers import (AutoModel, AutoTokenizer, AutoConfig,
                                  AutoModelForSequenceClassification, TrainingArguments, Trainer, logging)

In [15]:
print(torch.__version__)

2.0.1+cu118


In [16]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [17]:
print_gpu_utilization()

GPU memory occupied: 583 MB.


In [18]:
# импорт данных и первый их анализ
data = pd.read_csv(r"D:\toxic_comments.csv", index_col=0)
data.sample(10)

,text,toxic
127113,"Well, I have to agree that whoever destroyed t...",1
21330,"""\n I forgot to mention how long! I will give ...",0
44404,"""Hello , and welcome to Wikipedia! The first t...",0
66733,"""\n\n More information about the moschee \n\nM...",0
25350,"""\n\nFirst of all, where do you get the idea t...",0
140039,] // 10\n[http://www.webcitation.org/5oGeSq8ag 11,0
16995,I think Masem (whom I respect very highly) sum...,0
43630,"23:39, Oct 2, 2004 (UTC)",0
79459,Phil Donahue\n\nI know he was at WDTN during h...,0
14148,"""\nYeagh; self-auditing. This sort of demand ...",0


In [19]:
#получение общей информации о датасете
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [20]:
#описание данных
data.describe()

,toxic
count,159292.000000
mean,0.101612
std,0.302139
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


Можно говорить о дисбалансе классов. Около 10% токсичных комментариев, 90% не токсичны.

In [21]:
%%time
#загрузка модели и токенизатора BERT
model = AutoModel.from_pretrained("unitary/toxic-bert")
tokenizer = AutoTokenizer.from_pretrained('unitary/toxic-bert')

CPU times: total: 1.94 s
Wall time: 2.93 s


In [22]:
%%time
#проводим первичную токенизацию для текста
pre_tokenized = data['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

Token indices sequence length is longer than the specified maximum sequence length for this model (631 > 512). Running this sequence through the model will result in indexing errors


CPU times: total: 44.1 s
Wall time: 44.1 s


Как видим из сообщения, имеем ограничение в 512 токенов в строке. Необходимо убрать строки с превышением лимита.

In [23]:
#создаем ограничение
data_temp = pd.DataFrame(pre_tokenized)
data_temp['toxic'] = data['toxic']
len_list = []
for i in data_temp['text']:
    len_list.append(len(i))
data_temp['len'] = len_list
data_temp.head()

,text,toxic,len
0,"[101, 7526, 2339, 1996, 10086, 2015, 2081, 210...",0,68
1,"[101, 1040, 1005, 22091, 2860, 999, 2002, 3503...",0,35
2,"[101, 4931, 2158, 1010, 1045, 1005, 1049, 2428...",0,54
3,"[101, 1000, 2062, 1045, 2064, 1005, 1056, 2191...",0,144
4,"[101, 2017, 1010, 2909, 1010, 2024, 2026, 5394...",0,21


In [24]:
#удаляем строки с превышающими лимит токенами
data_temp = data_temp.loc[data_temp['len'] < 513].drop(['len'], axis=1)

In [25]:
#собираем датафрейм после фильтрации
data_left = data['text']
data_right = data_temp['toxic']
data_final = pd.concat([data_left, data_right], axis=1, join="inner")
data_final.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [26]:
#для получения embedding для всего датасета потребуются большие вычислительные мощности, поэтому процедуру их расчета представил на выборке в 311 
out, data_final = train_test_split(data_final, test_size=0.999, #с соотношением 
    random_state=12345, #с заданной опорой для рандома 
    stratify= data_final['toxic']) #с заданной стратификацией по целевому признаку
data_final = data_final.reset_index(drop=True)
data_final.shape

(155634, 2)

In [27]:
data_final.head()

,text,toxic
0,"""\n\nI believe he archived the discussion beca...",0
1,Bot error \n\nOrphanBot applied an incorrect t...,0
2,Truce\n\nI call a truce. But I didn't see thi...,0
3,It's unfortunate that the sources are vague on...,0
4,it staying and no buts about it\n\ni wll keep ...,0


In [28]:
#проверяем баланс классов
print(round((data_final['toxic'].count() - data_final['toxic'].sum()) / data_final['toxic'].count() * 100, 2), '%')

89.83 %


Баланс классов соблюдается

In [29]:
%%time
#проводим токенизацию
tokenized = data_final['text'].apply(
  lambda x: tokenizer.encode(x, add_special_tokens=True))

CPU times: total: 39.4 s
Wall time: 39.5 s


In [30]:
#проверяем длину токенов
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
max_len

512

In [31]:
#паддинг и маска
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)

In [32]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
model.eval()


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [33]:
%%time
#создание эмбеддингов
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).to(device)
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).to(device)
        
    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

  0%|          | 0/1556 [00:00<?, ?it/s]

CPU times: total: 1h 59min 11s
Wall time: 1h 43min 55s


In [34]:
#создание датасета с эмбеддингами и таргетом
data_ed = pd.DataFrame(np.concatenate(embeddings)) 
data_ed['toxic'] = data_final['toxic']
data_ed

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,toxic
0,-0.579122,-0.976771,0.526930,-0.344545,1.073406,0.116593,-0.191745,0.019543,-0.462766,-0.718300,...,-1.112387,0.175781,-0.709351,0.126624,0.879271,-0.525642,-0.726852,0.379305,0.087888,0
1,-0.657009,-1.018494,0.449042,-0.272752,0.991285,0.317469,-0.043647,0.013462,-0.390273,-0.622839,...,-1.212102,0.227788,-0.635808,0.140072,0.852536,-0.453627,-0.715883,0.316769,0.284531,0
2,-0.754768,-0.931944,0.428935,-0.719045,0.914379,0.537290,0.065258,-0.194789,-0.445472,-0.878202,...,-1.210364,0.502389,-0.651083,0.111464,0.670703,-0.544642,-0.625255,0.514120,-0.026531,0
3,-0.638190,-0.966847,0.419287,-0.533452,0.997320,0.215355,-0.146769,0.056217,-0.409464,-0.591199,...,-1.233097,0.209634,-0.693372,0.049386,0.884811,-0.568885,-0.804295,0.446663,0.035812,0
4,-0.427659,-0.665191,0.925915,-0.578008,0.789994,0.310947,0.231700,0.144695,-0.182835,-0.503202,...,-1.040302,0.513854,-0.360468,0.196113,0.722595,-0.459235,-0.981930,0.709400,0.314995,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155595,-0.758716,-0.910621,0.399189,-0.492608,0.946495,0.246570,-0.092683,0.047783,-0.560727,-0.747882,...,-1.275620,0.291381,-0.676891,0.438761,0.945436,-0.589925,-0.865059,0.281674,0.140597,0
155596,-0.717656,-0.941628,0.397711,-0.228470,1.182325,0.274734,0.326376,0.139614,-0.160068,-0.297057,...,-0.617464,0.499630,-0.541272,-0.495913,0.408830,-0.475904,-0.771808,0.697678,0.467036,0
155597,-0.701214,-0.901648,0.792093,-0.387406,1.077377,0.299192,-0.080067,0.024048,-0.431653,-0.699643,...,-1.285836,0.286392,-0.677304,0.148410,0.758705,-0.524568,-0.678514,0.331908,0.126121,0
155598,-0.589732,-0.943831,0.565660,-0.325469,1.039689,0.185136,-0.104915,0.097853,-0.356026,-0.691238,...,-1.184709,0.097172,-0.826426,0.114271,0.928049,-0.575514,-0.806420,0.302182,0.042963,0


In [35]:
#сохранение эмбеддингов в файл
data_ed.to_csv(r'D:\toxic_bert_full_toxic_data_ed.csv', index=False)

In [36]:
#загружаем ранее преобразованные эмбеддинги из файла
data_embedded = pd.read_csv(r'D:\toxic_bert_full_toxic_data_ed.csv', index_col=0)
data_embedded.head()

,1,2,3,4,5,6,7,8,9,10,...,759,760,761,762,763,764,765,766,767,toxic
0,,,,,,,,,,,,,,,,,,,,,
-0.579122,-0.976771,0.526930,-0.344545,1.073406,0.116593,-0.191745,0.019543,-0.462765,-0.718299,-0.240531,...,-1.112387,0.175781,-0.709351,0.126624,0.879271,-0.525642,-0.726853,0.379305,0.087888,0
-0.657009,-1.018494,0.449042,-0.272752,0.991285,0.317469,-0.043647,0.013462,-0.390273,-0.622839,-0.202760,...,-1.212102,0.227788,-0.635808,0.140072,0.852536,-0.453627,-0.715883,0.316769,0.284531,0
-0.754768,-0.931944,0.428935,-0.719045,0.914380,0.537290,0.065258,-0.194789,-0.445472,-0.878202,-0.072412,...,-1.210364,0.502389,-0.651083,0.111464,0.670703,-0.544642,-0.625255,0.514120,-0.026531,0
-0.638190,-0.966847,0.419287,-0.533452,0.997320,0.215355,-0.146769,0.056217,-0.409464,-0.591199,-0.214927,...,-1.233097,0.209634,-0.693372,0.049386,0.884811,-0.568885,-0.804295,0.446663,0.035812,0
-0.427659,-0.665191,0.925914,-0.578008,0.789994,0.310947,0.231700,0.144695,-0.182835,-0.503202,-0.042319,...,-1.040302,0.513854,-0.360468,0.196113,0.722595,-0.459235,-0.981930,0.709400,0.314995,0


In [37]:
data_embedded.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 155600 entries, -0.57912236 to -0.6520197
Columns: 768 entries, 1 to toxic
dtypes: float64(767), int64(1)
memory usage: 912.9 MB


In [38]:
#дисбаланс классов сохранен
print(round((data_embedded['toxic'].count() - data_embedded['toxic'].sum()) / data_embedded['toxic'].count() * 100, 2), '%')

89.83 %


## Обучение

### Разбиение на выборки

In [39]:
features_train, features_test, target_train, target_test = train_test_split( #создаем 4 датасета, два признаков и два целевых, 
    data_embedded.drop(columns='toxic'), #для датасетов признаков удаляем целевой
    data_embedded['toxic'], #для целевого оставляем только целевой
    test_size=0.2, #с соотношением 
    random_state=12345, #с заданной опорой для рандома 
    stratify= data_embedded['toxic']) #с заданной стратификацией по целевому признаку

### Выбор наилучшей модели и поиск гиперпараметров через GridSearch

In [40]:
#подбор модели через пайплайн
pipeline = Pipeline([('model', LogisticRegression())])
pipeline

Pipeline(steps=[('model', LogisticRegression())])

In [41]:
#создание перечня для перебора параметров
estimators_range = [x for x in range(1, 30, 2)]
max_depth_range = [x for x in range(20, 1000, 20)]

In [42]:
#формирование набора параметров
params = [{
        'model': [LogisticRegression(solver='liblinear', random_state=12345, class_weight= 'balanced')],
        'model__max_iter': range (100, 1000, 100),
        'model__C': np.arange(0.1, 1.0, 0.1),
    },
        
    
    {
        'model': [RandomForestClassifier(class_weight= 'balanced', random_state=12345)],
        'model__n_estimators': estimators_range,
        'model__max_depth': max_depth_range,
         },

    {
        'model': [DecisionTreeClassifier(class_weight= 'balanced', random_state=12345)],
        'model__max_depth': max_depth_range,
            }

]

In [43]:
#модель для RandomizedSearchCV
grid = RandomizedSearchCV(pipeline,
                    params,
                    cv=5,
                    verbose=1,
                    random_state=12345,
                    scoring='f1',
                    n_jobs=-1)

In [44]:
%%time
#обучаем модель на тренировочных данных
grid.fit(features_train, target_train.values)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
CPU times: total: 4min 28s
Wall time: 43min 43s


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('model', LogisticRegression())]),
                   n_jobs=-1,
                   param_distributions=[{'model': [LogisticRegression(class_weight='balanced',
                                                                      random_state=12345,
                                                                      solver='liblinear')],
                                         'model__C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                                         'model__max_iter': range(100, 1000, 100)},
                                        {'model': [RandomForestClassifier(class_weight='b...
                                                              520, 540, 560,
                                                              580, 600, ...],
                                         'model__n_estimators': [1, 3, 5, 7, 9,
                                                                 11, 13, 15, 17,
                                                                 19, 21, 23, 25,
                                                                 27, 29]},
                                        {'model': [DecisionTreeClassifier(class_weight='balanced',
                                                                          random_state=12345)],
                                         'model__max_depth': [20, 40, 60, 80,
                                                              100, 120, 140,
                                                              160, 180, 200,
                                                              220, 240, 260,
                                                              280, 300, 320,
                                                              340, 360, 380,
                                                              400, 420, 440,
                                                              460, 480, 500,
                                                              520, 540, 560,
                                                              580, 600, ...]}],
                   random_state=12345, scoring='f1', verbose=1)

In [45]:
#выводим параметры наилучшей модели
grid.best_params_

{'model__n_estimators': 21,
 'model__max_depth': 400,
 'model': RandomForestClassifier(class_weight='balanced', max_depth=400, n_estimators=21,
                        random_state=12345)}

In [46]:
#выводим лучший результат
abs(grid.best_score_)

0.9392359185464443

Лучшие параметры у модели RandomForestClassifier(class_weight='balanced', max_depth=400, n_estimators=21,
                        random_state=12345). Лучший результат f1=0.939

### Модель CatBoost

In [47]:
%%time
#Обучаем модель CatBoostRegressor
cb = CatBoostClassifier(custom_metric='F1', eval_metric='F1', iterations=50)
cb.fit(features_train, target_train.values, verbose=5)
print(f'F1: {cb.best_score_}')

Learning rate set to 0.5
0:	learn: 0.9302775	total: 459ms	remaining: 22.5s
5:	learn: 0.9487665	total: 1.25s	remaining: 9.2s
10:	learn: 0.9533948	total: 2.03s	remaining: 7.21s
15:	learn: 0.9566898	total: 2.79s	remaining: 5.92s
20:	learn: 0.9607797	total: 3.56s	remaining: 4.92s
25:	learn: 0.9636148	total: 4.36s	remaining: 4.02s
30:	learn: 0.9661439	total: 5.13s	remaining: 3.14s
35:	learn: 0.9686846	total: 5.93s	remaining: 2.31s
40:	learn: 0.9713811	total: 6.68s	remaining: 1.47s
45:	learn: 0.9728875	total: 7.45s	remaining: 648ms
49:	learn: 0.9740162	total: 8.04s	remaining: 0us
F1: {'learn': {'Logloss': 0.018272324882952277, 'F1': 0.9740162151473205}}
CPU times: total: 1min 9s
Wall time: 11.2 s


Модель CatBoost показывает лучший результат в 'F1': 0.974

### Модель LightGBM

In [48]:
%%time

lgbm = lgb.LGBMClassifier(class_weight= 'balanced', random_state=12345) #
parameters = {'n_estimators': estimators_range, 'max_depth': max_depth_range, 'learning_rate': np.arange(0.05, 0.5, 0.05)}
#применение метода гридсёрч со встроенной кросс-валидацией

rand_lgbm = RandomizedSearchCV(lgbm, n_iter=20, param_distributions= parameters, scoring='f1', n_jobs= -1, cv=5, random_state=12345)
#обучение модели
rand_lgbm.fit(features_train, target_train.values)

#лучшее значение после перебора параметров 
best_lgbm = rand_lgbm.best_score_

print("Лучшие параметры для модели LGBMClassifier с "\
    "использованием кросс-валидации:", rand_lgbm.best_params_)
print("Наибольшее значение метрики F1 для модели LGBMClassifier "\
    "при лучших гиперпараметрах с использованием кросс-валидации:", best_lgbm)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 12659, number of negative: 111821
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.750040 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 195585
[LightGBM] [Info] Number of data points in the train set: 124480, number of used features: 767
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
Лучшие параметры для модели LGBMClassifier с использованием кросс-валидации: {'n_estimators': 25, 'max_depth': 740, 'learning_rate': 0.25}
Наибольшее значение метрики F1 для модели LGBMClassifier при лучших гиперпараметрах с использованием 

Для модели LGBMClassifier лучшие параметры {'n_estimators': 25, 'max_depth': 740, 'learning_rate': 0.25}
F1 : 0.930 

Результаты работы моделей представлены ниже.
- Лучшие параметры у модели RandomForestClassifier(class_weight='balanced', max_depth=400, n_estimators=21, random_state=12345). Лучший результат f1=0.939
- Модель CatBoost показывает лучший результат в 'F1': 0.974
- Для модели LGBMClassifier лучшие параметры {'n_estimators': 25, 'max_depth': 740, 'learning_rate': 0.25}. F1 : 0.930 

Исходя из этого, наилучшей моделью для предсказания токсичности является CatBoost

## Тестирование

In [49]:
%%time
#Тестируем модель CatBoostRegressor
pred_cat_test = cb.predict(features_test)
print(f'F1:', f1_score(target_test, pred_cat_test))

F1: 0.9406940063091482
CPU times: total: 297 ms
Wall time: 680 ms


На тестовой выборке результат позитивный. F1: 0.941.
Метрика выше 0,75

## Выводы

- проведен анализ датасета. Выявлен дисбаланс. Около 10% токсичных комментариев, 90% не токсичны;
- загрузили модель и токенизатор BERT;
- проводим первичную токенизацию для текста. Выявили ограничение длины в 512 токенов. Очистили датасет от длинных комментариев.
- создали эмбеддинги на выборке в 15900 строк
- разбили выборку на тренировочныую и тестовую
- обучили модели. Результаты работы моделей представлены ниже.
  - Лучшие параметры у модели RandomForestClassifier(class_weight='balanced', max_depth=400, n_estimators=21, random_state=12345). Лучший результат f1=0.939
  - Модель CatBoost показывает лучший результат в 'F1': 0.974
  - Для модели LGBMClassifier лучшие параметры {'n_estimators': 25, 'max_depth': 740, 'learning_rate': 0.25}. F1 : 0.930

    Исходя из этого, наилучшей моделью для предсказания токсичности является CatBoost